# Uncertainty propagation

This notebook demonstrates uncertainty propagation with PyCO2SYS.

In this example, we imagine that we've measured dissolved inorganic carbon (DIC) and Total scale pH for four seawater samples.  The DIC and pH values are different from each other, but all other conditions are the same for the samples.  We want to know the seawater <i>p</i>CO<sub>2</sub> in the samples.

The first step is to define the input data and run `CO2SYS` on the data as normal:

In [1]:
import numpy as np
import PyCO2SYS as pyco2

# Set up CO2SYS inputs
par1 = np.array([2150.1, 2175.3, 2199.8, 2220.4])  # dissolved inorganic carbon in μmol/kg
par2 = np.array([ 8.121,  8.082,  8.041,  8.001])  # pH on the Total scale
par1type = 2
par2type = 3
sal = 33.1
tempin = 25.0
tempout = 25.0
presin = 0.0
presout = 0.0
si = 5.0
phos = 1.0
nh3 = 0.5
pHscale = 1
k1k2constants = 12
kso4constants = 1

# Run CO2SYS and print out the pCO2 values
co2dict = pyco2.CO2SYS(par1, par2, par1type, par2type, sal, tempin, tempout, presin, presout,
                       si, phos, pHscale, k1k2constants, kso4constants, NH3=nh3)
print("pCO2: ", co2dict["pCO2in"])

pCO2:  [352.02854204 393.52570303 441.56699187 492.87152363]


## Uncertainties from measurements

### Constant uncertainties

First, we want to know how the uncertainties in our DIC and pH measurements propagate into calculated <i>p</i>CO<sub>2</sub>.  Let's assume that the uncertainties in DIC and pH are (1) independent from each other and (2) constant for all measurements, with a one-standard-deviation uncertainty of 3 μmol/kg for DIC, and 0.005 for pH.

We need to assemble:

  1. A list of parameters that we want to propagate uncertainties into: `uncertainties_into`.
  2. A dict of parameters that we want to propagate uncertaintes from, and their uncertainties: `uncertainties_from`.
  
For both of these, the entries in the list (or keys in the dict) come from the corresponding fields in the [CO2SYS output dict](https://pyco2sys.readthedocs.io/en/latest/co2sys/#outputs).

**The uncertainties must be *independent* from each other and given in terms of *standard deviations* for the maths to work!**

In [2]:
# Which parameter(s) do we want to propagate uncertainties into?
uncertainties_into = ["pCO2in"]

# Define measurement uncertainties
uncertainties_from = {
    "PAR1": 3,
    "PAR2": 0.005,
}

# Propagate and print out results
uncertainties, components = pyco2.uncertainty.propagate(co2dict, uncertainties_into, uncertainties_from)
print("Uncertainty in pCO2 from PAR1 (i.e. DIC):     ", components["pCO2in"]["PAR1"])
print("Uncertainty in pCO2 from PAR2 (i.e. pH):      ", components["pCO2in"]["PAR2"])
print("Total uncertainty in pCO2 from PAR1 and PAR2: ", uncertainties["pCO2in"])

Uncertainty in pCO2 from PAR1 (i.e. DIC):      [0.49117977 0.54271922 0.60219155 0.66592261]
Uncertainty in pCO2 from PAR2 (i.e. pH):       [4.52326826 5.01227633 5.57519448 6.17290103]
Total uncertainty in pCO2 from PAR1 and PAR2:  [4.54985861 5.04157299 5.60762233 6.20871646]


The output `uncertainties` is a dict where the keys are the list entries from `uncertainties_into`.  Its values are the total uncertainty in the corresponding variable.  For example, `uncertainties["pCO2in"]` contains the total uncertainty in calculated <i>p</i>CO<sub>2</sub> at input conditions (i.e. `"pCO2in"`).

The output `components` is a dict where the keys are the same as in `uncertainties`.  Here each value is itself a dict, with the keys taken from `uncertainties_from`.  Each inner value is the the component of uncertainty from the corresponding parameter.  For example, `components["pCO2in"]["PAR1"]` contains the uncertainty in calculated <i>p</i>CO<sub>2</sub> at input conditions (i.e. `"pCO2in"`) resulting from uncertainty in `PAR1` (here, DIC).

We can see from the results above that the uncertainty in pH dominates the overall uncertainty in <i>p</i>CO<sub>2</sub>.

### Different uncertainties for different samples

Now imagine that the first pH measurement was done with a more accurate method than the others, so its uncertainty is smaller.  We can account for this by using an array instead of a single value in the `uncertainties_from` dict.

You could also use this approach if `"PAR1"` and/or `"PAR2"` contained a mixture of different parameter types, with different uncertainties.

In [3]:
# Redefine measurement uncertainties
uncertainties_from = {
    "PAR1": 3,
    "PAR2": np.array([0.001, 0.005, 0.005, 0.005]),
}

# Re-propagate and print out results
uncertainties, components = pyco2.uncertainty.propagate(co2dict, uncertainties_into, uncertainties_from)
print("Uncertainty in pCO2 from PAR1 (i.e. DIC):     ", components["pCO2in"]["PAR1"])
print("Uncertainty in pCO2 from PAR2 (i.e. pH):      ", components["pCO2in"]["PAR2"])
print("Total uncertainty in pCO2 from PAR1 and PAR2: ", uncertainties["pCO2in"])

Uncertainty in pCO2 from PAR1 (i.e. DIC):      [0.49117977 0.54271922 0.60219155 0.66592261]
Uncertainty in pCO2 from PAR2 (i.e. pH):       [0.90465365 5.01227633 5.57519448 6.17290103]
Total uncertainty in pCO2 from PAR1 and PAR2:  [1.02939584 5.04157299 5.60762233 6.20871646]


The lower uncertainty in the first pH measurement has significantly reduced the total uncertainty in calculated <i>p</i>CO<sub>2</sub>.

### Other function inputs

We can include any of the non-settings inputs of `CO2SYS`, including the optional `NH3` and `H2S` inputs, by extending the `uncertainties_from` dict in the same way.  For example, to also get the uncertainty in aragonite saturation state, and to also propagate uncertainties in the temperature (of 0.05 °C) and salinity (of 0.002) values:

In [4]:
# Redefine which parameter(s) we want to propagate uncertainties into
uncertainties_into = ["pCO2in", "OmegaARin"]

# Redefine measurement uncertainties
uncertainties_from = {
    "PAR1": 3,
    "PAR2": np.array([0.001, 0.005, 0.005, 0.005]),
    "TEMPIN": 0.05,
    "SAL": 0.002,
}

# Re-propagate and print out results
uncertainties, components = pyco2.uncertainty.propagate(co2dict, uncertainties_into, uncertainties_from)
print("Uncertainty in pCO2 from temperature:                 ", components["pCO2in"]["TEMPIN"])
print("Uncertainty in Omega-aragonite from salinity:         ", components["OmegaARin"]["SAL"])
print("Total uncertainty in Omega-aragonite from all inputs: ", uncertainties["OmegaARin"])

Uncertainty in pCO2 from temperature:                  [0.0703395  0.07095754 0.07114568 0.07078549]
Uncertainty in Omega-aragonite from salinity:          [0.00017327 0.00016408 0.00015447 0.00014513]
Total uncertainty in Omega-aragonite from all inputs:  [0.01335874 0.04120918 0.03868048 0.03624898]


## Uncertainties from internal parameters

We can also determine how uncertainties in CO2SYS's internally evaluated parameters (totals from salinity and equilibrium constants) propagate through to the calculated results.  The approach is very similar to that used above: we just need to find the name of the variable we want to propagate uncertainty from in the `CO2SYS` output dict.

### Absolute uncertainty known

First, imagine that we know the absolute uncertainty in an equilibrium constant.  Let's say we think the uncertainty in <i>K</i><sub>1</sub> (first dissociation constant of carbonic acid) is on the order of 10<sup>-8</sup> (as a standard deviation).  The approach is now virtually identical to that used above:

In [5]:
# Add K1 uncertainty to the dict
uncertainties_from.update({"K1input": 1e-8})

# Re-propagate and print out results
uncertainties, components = pyco2.uncertainty.propagate(co2dict, uncertainties_into, uncertainties_from)
print("Uncertainty in pCO2 from K1:               ", components["pCO2in"]["K1input"])
print("Total uncertainty in pCO2 from all inputs: ", uncertainties["pCO2in"])
print("Uncertainty in Omega-aragonite from K1:               ", components["OmegaARin"]["K1input"])
print("Total uncertainty in Omega-aragonite from all inputs: ", uncertainties["OmegaARin"])

Uncertainty in pCO2 from K1:                [2.47722398 2.7678732  3.1040039  3.46253427]
Total uncertainty in pCO2 from all inputs:  [2.68352974 5.75184515 6.40979405 7.10932271]
Uncertainty in Omega-aragonite from K1:                [0.0001379  0.00014233 0.00014672 0.00015063]
Total uncertainty in Omega-aragonite from all inputs:  [0.01335945 0.04120942 0.03868076 0.03624929]


Above we see that this uncertainty in <i>K</i><sub>1</sub> would dominate the total uncertainty in <i>p</i>CO<sub>2</sub>, but be less important for the aragonite saturation state.

### Fractional uncertainty known

If you know the fractional uncertainty but not an absolute value, then you could just extract the relevant parameter values from the `co2dict` and multiply through.  For example, if we thought the uncertainty in <i>K</i><sub>2</sub> (second dissociation constant of carbonic acid) was on the order of 5%:

In [6]:
# Get K2 values from the co2dict
k2in = co2dict["K2input"]

# Add K2 uncertainty to the dict
uncertainties_from.update({"K2input": k2in * 0.05})

# Re-propagate and print out results
uncertainties, components = pyco2.uncertainty.propagate(co2dict, uncertainties_into, uncertainties_from)
print("Uncertainty in pCO2 from K2:               ", components["pCO2in"]["K2input"])
print("Total uncertainty in pCO2 from all inputs: ", uncertainties["pCO2in"])
print("Uncertainty in Omega-aragonite from K2:               ", components["OmegaARin"]["K2input"])
print("Total uncertainty in Omega-aragonite from all inputs: ", uncertainties["OmegaARin"])

Uncertainty in pCO2 from K2:                [2.12543169 2.19369936 2.26130983 2.32159417]
Total uncertainty in pCO2 from all inputs:  [3.42327208 6.15597592 6.79698329 7.47878793]
Uncertainty in Omega-aragonite from K2:                [0.18323862 0.17296734 0.16232941 0.15208668]
Total uncertainty in Omega-aragonite from all inputs:  [0.18372498 0.17780865 0.16687432 0.15634696]


In this example, we can see that the <i>K</i><sub>2</sub> uncertainty has a much bigger influence on aragonite saturation state than <i>K</i><sub>1</sub> did.

## Covarying uncertainties

If your uncertainties are covarying then you need to assemble a variance-covariance matrix of the uncertainties and a Jacobian matrix for the parameters that you are interested in.  The maths of how to do this are beyond the scope of this tutorial, but you can use PyCO2SYS to get all the derivatives that you need:

In [7]:
# Get raw derivatives
co2derivs, dxs = pyco2.uncertainty.forward(co2dict, uncertainties_into, uncertainties_from)

# Print out example results
print("   d(pCO2in)/d(PAR1)    = ", co2derivs["pCO2in"]["PAR1"])
print("d(OmegaARin)/d(K1input) = ", co2derivs["OmegaARin"]["K1input"])

   d(pCO2in)/d(PAR1)    =  [0.16372659 0.18090641 0.20073052 0.2219742 ]
d(OmegaARin)/d(K1input) =  [13790.4646484  14233.40688667 14672.08440132 15063.22765762]


The output `co2derivs` is a nested dict with a similar structure to `uncertainties`.